Using pdf miner

In [1]:
pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 51.1 MB/s eta 0:00:00


In [2]:
import pdfminer

In [3]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import HTMLConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import BytesIO

In [4]:
def converting_pdf_to_html(pdf_file):
    # Creating a PDF resource manager
    rsrcmgr = PDFResourceManager()
    # Creating a buffer to store the HTML content
    retstr = BytesIO()
    codec = 'utf-8'
    # Setting layout analysis parameters
    laparams = LAParams(char_margin=10.0, word_margin=5.0)
    # Creating an HTMLConverter
    device = HTMLConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    with open(pdf_file, 'rb') as pdf:
        pdf_interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.get_pages(pdf):
            pdf_interpreter.process_page(page)  # Processing each page of the PDF
    device.close()  # Closing the HTMLConverter device
    content = retstr.getvalue()  # HTML content from the buffer
    retstr.close()  # Closing the buffer

    # Returning the HTML content as a string
    return content.decode('utf-8')


In [5]:
pdf_file = '/content/Xinni_Chng.pdf'

In [6]:
html_resume = converting_pdf_to_html(pdf_file)

In [7]:
from IPython.display import HTML
HTML(html_resume)

In [8]:
html_resume

'<html><head>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n</head><body>\n<span style="position:absolute; border: gray 1px solid; left:0px; top:50px; width:2480px; height:3508px;"></span>\n<div style="position:absolute; top:50px;"><a name="1">Page 1</a></div>\n<div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:981px; top:332px; width:277px; height:36px;"><span style="font-family: SofiaProBold; font-size:36px">EXPERIENCE\n<br></span></div><div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:981px; top:492px; width:859px; height:101px;"><span style="font-family: Graphik-Bold; font-size:45px">Google</span><span style="font-family: MinionPro-Regular; font-size:48px"> </span><span style="font-family: Graphik-Regular; font-size:45px">— User experience Design intern\n<br></span><span style="font-family: Graphik-Regular; font-size:30px">MAY 2016 - PRESENT\n<br></span></div><div style="position:absolute; bord

Parsing the html using BeautifulSoup

In [9]:
pip install beautifulsoup4

In [10]:
from bs4 import BeautifulSoup
# Parsing the HTML content using BeautifulSoup
soup = BeautifulSoup(html_resume, 'html.parser')

In [11]:
soup

<html><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
</head><body>
<span style="position:absolute; border: gray 1px solid; left:0px; top:50px; width:2480px; height:3508px;"></span>
<div style="position:absolute; top:50px;"><a name="1">Page 1</a></div>
<div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:981px; top:332px; width:277px; height:36px;"><span style="font-family: SofiaProBold; font-size:36px">EXPERIENCE
<br/></span></div><div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:981px; top:492px; width:859px; height:101px;"><span style="font-family: Graphik-Bold; font-size:45px">Google</span><span style="font-family: MinionPro-Regular; font-size:48px"> </span><span style="font-family: Graphik-Regular; font-size:45px">— User experience Design intern
<br/></span><span style="font-family: Graphik-Regular; font-size:30px">MAY 2016 - PRESENT
<br/></span></div><div style="position:absolute; border: t

Finding the font sizes of the required headings

In [12]:
def find_specific_heading_style(html_resume, headings):
  '''Function to find the font style of the heading'''
  # Parsing the HTML content
  soup = BeautifulSoup(html_resume, 'html.parser')
  heading_style = {}

  # Iterating through all the <span> elements found in the HTML content
  for each_span in soup.find_all('span'):
        # Checking if the span contains any of the specified heading texts in a case-insensitive manner
        span_text = each_span.get_text().strip().lower()
        for heading in headings:
            if heading in span_text:
                # Extracting the font size
                span_style = each_span.get('style')
                heading_style[heading]=span_style
  return heading_style

In [13]:
# List of heading texts to search for
headings = ['education', 'experience', 'skill','award','project','achievement','internhsip','activities','link','certification','language']

In [14]:
# Calling the function to find and print font sizes for the specified headings
headings_style  = find_specific_heading_style(html_resume, headings)
headings_style

{'experience': 'font-family: Graphik-Regular; font-size:38px',
 'education': 'font-family: SofiaProBold; font-size:36px',
 'skill': 'font-family: SofiaProBold; font-size:36px',
 'award': 'font-family: Graphik-RegularItalic; font-size:38px',
 'project': 'font-family: SofiaProBold; font-size:36px'}

Finding the text of the education heading

In [15]:
education_style = headings_style['education']
education_style

'font-family: SofiaProBold; font-size:36px'

In [16]:
dict_instead_of_education = {key:value for key,value in headings_style.items() if key!='education'}
dict_instead_of_education

{'experience': 'font-family: Graphik-Regular; font-size:38px',
 'skill': 'font-family: SofiaProBold; font-size:36px',
 'award': 'font-family: Graphik-RegularItalic; font-size:38px',
 'project': 'font-family: SofiaProBold; font-size:36px'}

In [17]:
required_text = None
output = ''

for each_span in soup.find_all('span'):
    text = each_span.get_text().strip().lower()  # Convert text to lowercase for case-insensitive comparison
    span_style = each_span.get('style')
    text_style = None
    text_next = ""

    if span_style == education_style and 'education' in text:
        output += str(each_span)
        next_text = each_span.find_next('span')
        output += str(next_text)
        for key, value in dict_instead_of_education.items():

            # Check if the font size doesn't match and the key is not in the text_next
            if value != text_style and key not in text_next:
                #and each_span.find_parent('div') is not None and each_span.find_next_sibling() is None
                required_text = next_text.find_next('span')

                text_next = required_text.get_text().strip().lower()
                text_style = required_text.get('style')
                next_text=required_text
                output += str(required_text)
            else:
              continue


html_output = HTML(output)
display(html_output)